# YOLOv8 Training Tutorial

YOLOv8 is a computer vision model built by Ultralytics which support object detection, classification, key-point detection, and segmentation tasks.

In this tutorial we will fine-tunne the YOLOv8 nano model on a open source custom dataset to detect wood defects.

The dataset was made by Makar Baderko, and it is store on [Universe Roboflow](https://universe.roboflow.com/makar-baderko-kv1k2/plywood).


## Useful links
* [Dataset](https://universe.roboflow.com/makar-baderko-kv1k2/plywood/dataset/1)
* [YOLOv8 train documentation](https://docs.ultralytics.com/es/modes/train)


## Setup 🔧

In [ ]:
# Check the GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


We will use ClearML, an open source platform that automates machine learning experiment tracking.

In [ ]:
%pip install ultralytics --quiet
%pip install clearml --quiet

from ultralytics import YOLO
import clearml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.2/716.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.8 MB/s eta 0:00:00


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


### Download the dataset

Download the custom dataset from Roboflow and stores it in the folder `/datasets/`

Use the following dataset parameters:

```
APIKEY = "EtT3V7SooKlsIZMuIfRK"
WORKSPACE = "makar-baderko-kv1k2"
PROJECT = "plywood"
```

In [6]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

# Dataset parameters
APIKEY = "EtT3V7SooKlsIZMuIfRK"
WORKSPACE = "makar-baderko-kv1k2"
PROJECT = "plywood"

from roboflow import Roboflow
rf = Roboflow(api_key=APIKEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
dataset = project.version(1).download("yolov8")

%cd {HOME}

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.19, to fix: `pip install ultralytics==8.0.196`
/content


### Connect to Clear ML

Follow the steps to generate and add the key:

* Create a ClearML account in https://app.clear.ml.
* Go to [workspace-configuration](https://app.clear.ml/settings/workspace-configuration) and create new credentials
* Copy and paste the environment variables

In [1]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

%env CLEARML_API_ACCESS_KEY=YOUR_API_KEY
%env CLEARML_API_SECRET_KEY=YOUR_SECRET_KEY

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=YOUR_API_KEY
env: CLEARML_API_SECRET_KEY=YOUR_SECRET_KEY


## 🧑 Run the training

### Fix the data.yaml file

The `data.yaml` file from the dataset need two fixes:

* Add the path to the dataset
* fix the test, train and val paths

The following script overwrite the `data.yaml` file

In [8]:
with open('datasets/plywood-1/data.yaml', 'w') as f:
    f.write("""names:
- '0'
- '1'
- '2'
- '3'
- '4'
- '5'
- '6'
- '7'
nc: 8
roboflow:
  license: CC BY 4.0
  project: plywood
  url: https://universe.roboflow.com/makar-baderko-kv1k2/plywood/dataset/1
  version: 1
  workspace: makar-baderko-kv1k2
path: /content/datasets/plywood-1/
test: test/images
train: train/images
val: valid/images
""")


### Start the training

In [9]:
# Load the model.
model = YOLO('yolov8n.pt')

# Training.
results = model.train(
   data='datasets/plywood-1/data.yaml',
   imgsz=640,
   epochs=30,
   batch=8,
   name='my_yolov8n'
)


Ultralytics YOLOv8.1.19 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/plywood-1/data.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=my_yolov8n2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /content/datasets/plywood-1/train/labels.cache... 8685 images, 46 backgrounds, 0 corrupt: 100%|██████████| 8685/8685 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/datasets/plywood-1/valid/labels.cache... 540 images, 0 backgrounds, 0 corrupt: 100%|██████████| 540/540 [00:00<?, ?it/s]

Plotting labels to runs/detect/my_yolov8n2/labels.jpg... 



optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/my_yolov8n2
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G      3.076      5.434      2.308         29        640:   1%|          | 6/1086 [00:50<2:30:35,  8.37s/it]

2024-02-29 21:09:19,340 - clearml.log - WARNING - Event reporting sub-process lost, switching to thread based reporting


KeyboardInterrupt: 

## 🕵️‍♂️ Validate the custom model

In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data=datasets/plywood-1/data.yaml


## 🧪 Test the custom model

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source=datasets/plywood-1/test/images save=True


In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict3/*.jpg')[:3]:
      display(Image(filename=image_path, width=640))
      print("\n")